# SEMSCORE-DSPY

<img src="logo/SemScore-DSPy-2.svg" alt="SemScore-DSPy Logo" style="width:400px; height:auto;">


### Use SemScore Metric

#### DSPy

#### Pytorch

### Validation of SemScore Metric

Debug the `SemScoreValidator`

In [1]:
from semscore.utils.validation import SemScoreValidator
validator = SemScoreValidator(reference_file="semscore/data/reference/semscores_OA-100.json")
validator.debug_reference_file()

Successfully loaded reference file
Number of models: 2

Model: TinyLlama/TinyLlama-1.1B-Chat-v1.0
Number of cases: 154
First case keys: ['prompt', 'answer_ref', 'answer_pred', 'cosine_sim']

Model: models/Mistral-7B-Instruct-v0.2
Number of cases: 154
First case keys: ['prompt', 'answer_ref', 'answer_pred', 'cosine_sim']


Run validation to validate the SemScore metric against a reference: [Link](https://github.com/geronimi73/semscore/blob/main/nb_blog_part1_OA-conversations-results/semscores_OA-100.json).

Please see the validation results in the `validation_results` folder.

In [4]:
from semscore.utils.validation import run_validation
from IPython.display import Markdown, display


# Run validation with your dataset
report = run_validation(
    output_dir="validation_results",
    reference_file="semscore/data/reference/semscores_OA-100.json"
)

# The report will be saved to the output directory
# and returned as a string
display(Markdown(report))


# SemScore Validation Report
Generated on: 2024-12-21 21:39:34

## OpenAssistant Dataset Results

### TinyLlama/TinyLlama-1.1B-Chat-v1.0


Performance Metrics:
- Pearson Correlation: 1.0000
- Mean Absolute Difference: 0.0000
- Mean Calculated Score: 0.6974
- Mean Reference Score: 0.6974

Example Cases:
| Prompt | Prediction | Reference | Calculated | Expected | Diff |
|--------|------------|-----------|------------|-----------|------|
| <|user|> Can photoshop be used as a tool to forge ... | Yes, photoshop can be used as a tool to forge docu... | Yes, Photoshop can be used to forge documents due ... | 0.9537 | 0.9537 | 0.0000 |
| <|user|> What is the difference between C# and Jav... | Certainly! Here are some key differences between C... | Sure, I can provide you with a comparison between ... | 0.8042 | 0.8042 | 0.0000 |
| <|user|> Explain GPIO pins on a raspberry pi pico ... | GPIO (General Purpose Input/Output) pins are used ... | GPIO (General Purpose Input/Output) pins are physi... | 0.8529 | 0.8529 | 0.0000 |

### models/Mistral-7B-Instruct-v0.2


Performance Metrics:
- Pearson Correlation: 1.0000
- Mean Absolute Difference: 0.0000
- Mean Calculated Score: 0.7372
- Mean Reference Score: 0.7372

Example Cases:
| Prompt | Prediction | Reference | Calculated | Expected | Diff |
|--------|------------|-----------|------------|-----------|------|
| <s>[INST] Can photoshop be used as a tool to forge... | Yes, Adobe Photoshop and similar image editing sof... | Yes, Photoshop can be used to forge documents due ... | 0.8402 | 0.8402 | 0.0000 |
| <s>[INST] What is the difference between C# and Ja... | Both C# and Java are popular programming languages... | Sure, I can provide you with a comparison between ... | 0.9332 | 0.9332 | 0.0000 |
| <s>[INST] Explain GPIO pins on a raspberry pi pico... | The Raspberry Pi Pico W is a microcontroller board... | GPIO (General Purpose Input/Output) pins are physi... | 0.7874 | 0.7874 | 0.0000 |


In [6]:
from IPython.core.display import HTML

# HTML template for displaying images side by side
html_template = """
<div style="display: flex; justify-content: space-around; align-items: center;">
    <img src="{path1}" style="width: 45%; margin: 10px;">
    <img src="{path2}" style="width: 45%; margin: 10px;">
</div>
<div style="display: flex; justify-content: space-around; align-items: center;">
    <img src="{path3}" style="width: 45%; margin: 10px;">
    <img src="{path4}" style="width: 45%; margin: 10px;">
</div>
"""

display(HTML(html_template.format(
    path1="validation_results/score_dist_models_Mistral-7B-Instruct-v0.2_calculated_20241221_213932.png",
    path2="validation_results/score_dist_models_Mistral-7B-Instruct-v0.2_expected_20241221_213932.png",
    path3="validation_results/score_dist_TinyLlama_TinyLlama-1.1B-Chat-v1.0_calculated_20241221_213932.png",
    path4="validation_results/score_dist_TinyLlama_TinyLlama-1.1B-Chat-v1.0_expected_20241221_213932.png"
)))


Given the report and the histograms, it is fair to say that the SemScore metric produces the same calulated output as the reference implementation. 

### Characterization of SemScore Metric